In [2]:
from langchain.prompts import PromptTemplate
from langchain.chains import RetrievalQA
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.document_loaders import PyMuPDFLoader,DirectoryLoader
from langchain_community.document_loaders import PyPDFLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.prompts import PromptTemplate
from langchain.llms import CTransformers

In [3]:
# load Data from pdf 

def load_pdf(data):
   loader = DirectoryLoader(data,glob="*.pdf",loader_cls=PyPDFLoader)
   doc = loader.load()

   return doc

In [4]:
%pwd

'e:\\Medical-Bot'

In [14]:
ExtractData =load_pdf("Medical-Bot/Data/GaleEncyclopedia.pdf")

FileNotFoundError: Directory not found: 'Medical-Bot/Data/GaleEncyclopedia.pdf'